In [10]:
import yt
from astropy.table import Table
from astropy.io import ascii
from yt.analysis_modules.star_analysis.api import StarFormationRate

from utils.get_refine_box import get_refine_box
from get_halo_center import get_halo_center
import numpy as np

from enzoGalaxyProps import find_rvirial

import seaborn as sns


In [6]:
def _stars(pfilter, data):
    return data[(pfilter.filtered_type, "particle_type")] == 2

#this gets dark matter particles in zoom region only
def _darkmatter(pfilter, data):
    return data[(pfilter.filtered_type, "particle_type")] == 4

yt.add_particle_filter("stars",function=_stars, filtered_type='all',requires=["particle_type"])
yt.add_particle_filter("darkmatter",function=_darkmatter, filtered_type='all',requires=["particle_type"])


In [17]:
ds = yt.load('/Users/molly/foggie/halo_008508/natural/RD0020/RD0020')
ds.add_particle_filter('stars')
ds.add_particle_filter('darkmatter')

trackname = 'halo_tracks/008508/nref11n_selfshield_15/halo_track_200kpc_nref10'

yt : [INFO     ] 2019-09-16 14:03:50,677 Parameters: current_time              = 154.80103271343
yt : [INFO     ] 2019-09-16 14:03:50,678 Parameters: domain_dimensions         = [256 256 256]
yt : [INFO     ] 2019-09-16 14:03:50,680 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2019-09-16 14:03:50,681 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2019-09-16 14:03:50,683 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2019-09-16 14:03:50,684 Parameters: current_redshift          = 1.9999996880027
yt : [INFO     ] 2019-09-16 14:03:50,687 Parameters: omega_lambda              = 0.715
yt : [INFO     ] 2019-09-16 14:03:50,688 Parameters: omega_matter              = 0.285
yt : [INFO     ] 2019-09-16 14:03:50,689 Parameters: hubble_constant           = 0.695
Parsing Hierarchy : 100%|██████████| 994/994 [00:00<00:00, 11658.04it/s]
yt : [INFO     ] 2019-09-16 14:03:50,857 Gathering a field list (this may take a moment.)
/Users/molly/anacond

In [11]:

print("opening track: " + trackname)
track = Table.read(trackname, format='ascii')
track.sort('col1')
zsnap = ds.get_parameter('CosmologyCurrentRedshift')
comoving_box_size = ds.get_parameter('CosmologyComovingBoxSize')
print('Comoving Box Size:', comoving_box_size)
refine_box, refine_box_center, refine_width = get_refine_box(ds, zsnap, track)


opening track: halo_tracks/008508/nref11n_selfshield_15/halo_track_200kpc_nref10
Comoving Box Size: 100
inside get_refine_box 1.9999996880027
get_refine_box: using this location:      col1       col2     col3     col4     col5     col6     col7   col8
------------- -------- -------- -------- -------- -------- -------- ----
1.99979384142 0.493035 0.487894 0.501194 0.495035 0.489894 0.503194   10


In [15]:
search_radius = 10.
this_search_radius = search_radius / (1+ds.get_parameter('CosmologyCurrentRedshift'))
center, velocity = get_halo_center(ds, refine_box_center, radius=this_search_radius)


get_halo_centers:  code_length code_velocity
we have obtained the spherical region
we have extracted the DM density
 we have obtained the preliminary center
got the velocities
Located the main halo at: [0.4940366744995117, 0.4888944625854492, 0.5021944046020508] [-0.0025885967979688714 code_velocity, -0.003327571430154894 code_velocity, 2.636260457252462e-06 code_velocity]


In [18]:
rvir = find_rvirial(refine_box, ds, center)
# Mvir is mass within rvir
vir_sphere = ds.sphere(center,rvir)


	Now running fine search on the virial radius


In [44]:
vir_dm_mass = vir_sphere[('darkmatter', 'particle_mass')].in_units('Msun')
Mvir = vir_dm_mass.sum()


In [45]:
fgal = 0.07 ## fraction of the virial radius we are saying is the galaxy radius; totes made up
gal_sphere = ds.sphere(center, fgal*rvir)
# Mstar is sum of stellar mass; Mism is sum of gas mass; SFR is sum of SFR
gal_stars_mass = gal_sphere[('stars', 'particle_mass')].in_units('Msun')
Mstar = gal_stars_mass.sum()

gal_ism_mass = gal_sphere['cell_mass'].in_units('Msun')
Mism = gal_ism_mass.sum()

sfr = StarFormationRate(ds, data_source=gal_sphere)


In [46]:
gal_ism_mass.sum()

2423260105.658191 Msun

In [47]:
row = [zsnap, 'RD0020',
        center[0], center[1], center[2],
        rvir, Mvir, Mstar, Mism, sfr.Msol_yr[-1]]

In [62]:
gal_sphere

YTSphere (RD0020): , center=[7.31145347e+25 7.23535175e+25 7.43218310e+25] cm, radius=1.0583874102700775e+22 cm

In [55]:
t = Table(dtype=('f8','S6', 'f8', 'f8', 'f8', 
                'f8', 'f8', 'f8','f8', 'f8'),
                names=('redshift', 'name', 'xc', 'yc', 'zc',
                'rvir', 'Mvir', 'Mstar', 'Mism', 'SFR'))


In [56]:
t.add_row(row)

In [58]:
t['Mstar']/t['Mvir']

0.1278742437664437


In [67]:
ds.parameter_filename[-6:]

'RD0020'

In [72]:
haloinfo = 'halo_infos/008508/orig_nref11n/nref11n_nref10f/halo_info'
tt = ascii.read(haloinfo, format='fixed_width')
tt.sort('redshift')

In [84]:
output = 'RD0020'
thisid = tt['name'] ==  output
tt.colnames

['redshift', 'name', 'xc', 'yc', 'zc', 'Rvir', 'Mvir', 'Mstar', 'Mism', 'SFR']

In [86]:
len(tt[thisid])

1

In [87]:
from astropy.io import fits


In [98]:
fitsfile = '/Users/molly/Dropbox/foggie-collab/plots_halo_008508/orig_nref11n/nref11n_nref10f/spectra/lls/for_asdi/april2019/hlsp_misty_foggie_halo008508_nref11n_nref10f_rd0020_axz_dx072.9_dy036.6_v6_los.fits.gz'
hdu = fits.open(fitsfile)

In [93]:
zsnap = hdu[0].header['redshift']
thisid = tt['redshift'] ==  zsnap


In [99]:
hdu[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
AUTHOR  = 'Molly S. Peeples'                                                    
DATE    = '2018-08-21T11:46:12.204035'                                          
REDSHIFT=      1.9999996879495                                                  
RAYSTART= '0.494611705009, 0.488881383256, 0.501099'                            
RAYEND  = '0.494611705009, 0.488881383256, 0.503099'                            
SIM_NAME= 'Tempest_nref11n_nref10f'                                             
NLINES  = '6       '                                                            
DOI     = 'https://doi.org/10.3847/1538-4357/ab0654'                            
PAPER   = 'Peeples et al. (2